## This is the notebook for exploring data and prepaing distance matrix regarding SBS signature exposure for the five cancer types， based on Hartwig cohort

### Import modules

In [ ]:
# import modules 
import os
import time
import random
import pandas as pd
import numpy as np

# modules for calculation
import sklearn
import scipy
import scipy.stats as spst
import scipy.spatial as sp
import scipy.cluster.hierarchy as hc
from sklearn import metrics

# modules for plots
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# pyEMD module
import pyemd
from pyemd import emd
from pyemd import emd_with_flow

os.chdir('/path/to/EMD_project/')
os.getcwd()

# Versions of modules
print("scikit-learn version:", sklearn.__version__)
print("scipy version:", scipy.__version__)
print("pyemd version:", pyemd.__version__)
print('pandas version:', pd.__version__)
print('numpy version:', np.__version__)
print('seaborn version:', sns.__version__)
!python --version


#### Load related files: 
'Etiology Information of Signatures_SBS5_Unknown_20240527.csv' is generated by '_Prepare_Signatures_in_Samples_Sigs_Etiology_Info.ipynb';

These are the four files for the exposure of signatures in cancer samples from Hartwig cohort, which generated by Lancelot Seillier
- 'Hartwig_BRCA_Assignment_Solution_Activities_PCAWGref.txt',
- 'Hartwig_LUAD_Assignment_Solution_Activities_PCAWGref.txt',
- 'Hartwig_COREAD_Assignment_Solution_Activities_PCAWGref.txt',
- 'Hartwig_HCC_Assignment_Solution_Activities_PCAWGref.txt'

In [ ]:
###++++++++++++++++++++++++
# Extract the data of the five cancer types: 
#    Breast cancer, Lung-AdenoCa, ColoRect-AdenoCa and Liver-HCC
# Normalize the signatures exposure data to factions per each sample
###++++++++++++++++++++++++

SigEtioTable = pd.read_csv('Etiology Information of Signatures_SBS5_Unknown_20240527.csv', index_col='Unnamed: 0')
SigEtioTable

filenames = ['Hartwig_BRCA_Assignment_Solution_Activities_PCAWGref.txt',
             'Hartwig_LUAD_Assignment_Solution_Activities_PCAWGref.txt',
             'Hartwig_COREAD_Assignment_Solution_Activities_PCAWGref.txt',
             'Hartwig_HCC_Assignment_Solution_Activities_PCAWGref.txt'
             ]
cancer_types = ['Breast-cancer', 'Lung-AdenoCa', 'ColoRect-AdenoCa', 'Liver-HCC']

dict_allFrac = {} ## dict to accept prepared data

for cancer_type, filename in zip(cancer_types, filenames):
    ## load the file
    df = pd.read_csv(filename, sep='\t')

    df=df.rename(columns={'Samples': 'clinic_ID'})
    df=df.set_index('clinic_ID')
    df['Cancer_Types'] = cancer_type

    ## Move 'Cancer_Type' to the first position
    column_order = ['Cancer_Types'] + [col for col in df if col != 'Cancer_Types']
    df = df[column_order]
    
    ## Take the signatures
    Sigs = df.columns[1:]
    d0 = df.copy()
    
    ## 
    df_raw = d0.drop(['Cancer_Types'], axis = 1)

    ## Calculate the Fractions of mutational signatuters exposures in samples
    total_counts = df_raw.sum(axis=1)
    df_allFrac = df_raw.div(total_counts, axis=0)

    ## multi-labeling of samples according to the existences of certain class of signatures
    # Apobec signaitures
    df_allFrac['label_Apobec'] = df_allFrac.apply(lambda row: 'nonApobec' if (row['SBS2'] == 0 and row['SBS13'] == 0) 
                                                  else 'Apobec', axis=1)
    df_allFrac['signal_Apobec'] = df_allFrac['SBS2'] + df_allFrac['SBS13']

    # Mismatch Repair signatures
    df_allFrac['label_MMR'] = df_allFrac.apply(lambda row: 'nonMMR' if (row['SBS6'] == 0 
                                                                        and row['SBS14'] == 0 
                                                                        and row['SBS15'] == 0 
                                                                        and row['SBS20'] == 0 
                                                                        and row['SBS21'] == 0 
                                                                        and row['SBS26'] == 0 
                                                                        and row['SBS44'] == 0) 
                                                     else 'MMR', axis=1)
    df_allFrac['signal_MMR'] = df_allFrac['SBS6'] + df_allFrac['SBS14'] + df_allFrac['SBS15'] + df_allFrac['SBS20'] + df_allFrac['SBS21'] + df_allFrac['SBS26'] + df_allFrac['SBS44'] 

    # Tobacco signatures
    df_allFrac['label_Tobacco'] = df_allFrac.apply(lambda row: 'nonTobacco' if (row['SBS4'] == 0 
                                                                                and row['SBS29'] == 0) 
                                                   else 'Tobacco', axis=1)
    df_allFrac['signal_Tobacco'] = df_allFrac['SBS4'] + df_allFrac['SBS29']

    # UV signatures
    df_allFrac['label_UV'] = df_allFrac.apply(lambda row: 'nonUV' if (row['SBS7a'] == 0 
                                                                      and row['SBS7b'] == 0 
                                                                      and row['SBS7c'] == 0 
                                                                      and row['SBS7d'] == 0 
                                                                      and row['SBS38'] == 0) 
                                              else 'UV', axis=1)
    df_allFrac['signal_UV'] = df_allFrac['SBS7a'] + df_allFrac['SBS7b'] + df_allFrac['SBS7c'] + df_allFrac['SBS7d'] + df_allFrac['SBS38']

    # POLE signatures
    df_allFrac['label_POLE'] = df_allFrac.apply(lambda row: 'nonPOLE' if (row['SBS10a'] == 0 
                                                                        and row['SBS10b'] == 0) 
                                                 else 'POLE', axis=1)
    df_allFrac['signal_POLE'] = df_allFrac['SBS10a'] + df_allFrac['SBS10b']

    # Clock-likesignatures
    df_allFrac['label_ClockLike'] = df_allFrac.apply(lambda row: 'nonClockLike' if (row['SBS1'] == 0) 
                                                     else 'ClockLike', axis=1)
    df_allFrac['signal_ClockLike'] = df_allFrac['SBS1']

    # Base_Excision_Repair, BER
    df_allFrac['label_BER'] = df_allFrac.apply(lambda row: 'nonBER' if (row['SBS30'] == 0 
                                                                        and row['SBS36'] == 0) 
                                               else 'BER', axis=1)
    df_allFrac['signal_BER'] = df_allFrac['SBS18'] + df_allFrac['SBS30'] + df_allFrac['SBS36']

    # Platinum Treatment
    df_allFrac['label_Platinum'] = df_allFrac.apply(lambda row: 'nonPlatinum' if (row['SBS31'] == 0 
                                                                                  and row['SBS35'] == 0) 
                                                    else 'Platinum', axis=1)
    df_allFrac['signal_Platinum'] = df_allFrac['SBS31'] + df_allFrac['SBS35']

    ## Take a look of the data
    df_allFrac['Cancer_Types'] = df['Cancer_Types']
    
    dict_allFrac[cancer_type] = df_allFrac

dict_allFrac
dict_allFrac['Breast-cancer']

In [ ]:
### Check the sample size
for cancer_type in cancer_types:
    df = dict_allFrac[cancer_type]
    print(f'Sample size of {cancer_type}: {df.shape[0]}')

#### Save all the files of multiple layers labeled SBS signaturtes exposures in cancer samples, which used to provide labels info for clustering performance estimation

In [ ]:
### Save all the files of multi-labeld SBS signaturtes exposures in cancer samples

cancer_types = ['Breast-cancer', 'Lung-AdenoCa', 'ColoRect-AdenoCa', 'Liver-HCC']

## Dump the files and test it
for cancer_type in cancer_types:
    
    df_frac = dict_allFrac[cancer_type].copy()
    
    ## Write out the prepared Fraction data of mutational signatures
    df_frac.to_csv(f'_Hartwig {cancer_type} Mutational Signatures Fraction Multi-Label Data_exper01.csv', index=True)
    
    ## Load to check whether the files were correctly saved
    df_frac_test = pd.read_csv(f'_Hartwig {cancer_type} Mutational Signatures Fraction Multi-Label Data_exper01.csv', index_col=0)

df_frac_test


In [ ]:
### Prepare the ratio of different etiology labels in different cance types
dict_allFrac
cancer_types = ['Breast-cancer', 'Lung-AdenoCa', 'ColoRect-AdenoCa', 'Liver-HCC']
label_types = ['label_Apobec', 'label_MMR', 'label_Tobacco', 'label_UV', 'label_POLE', 'label_ClockLike', 'label_BER', 'label_Platinum'] ## type of labels


# Dictionary to store the ratios
all_ratios = {} ## dict for all results

for cancer_type in cancer_types: ## looping cancer types
    ## for each cancer type
    df_frac = dict_allFrac[cancer_type]
    
    ratios = {} ## dict to hold ratios for certain cancer type
    for column in label_types:
        # Count the occurrences of each category
        counts = df_frac[column].value_counts()

        # Calculate the ratio
        if len(counts) == 1:
            ratios[column] = 'One-Class'

        if len(counts) == 2:
            key1, key2 = counts.keys()
            if key1.startswith('non'):
                ratio = counts[key2] / counts[key1]
            else:
                ratio = counts[key1] / counts[key2]

            # Store the ratio in the dictionary
            ratios[column] = round(ratio, 3)
    ## collect the ratios
    all_ratios[cancer_type] = ratios
    
dict_all_ratios = all_ratios.copy()

for cancer_type in cancer_types:
    print(cancer_type, dict_all_ratios[cancer_type])

### Draw the Proportions of Existence of SBS signatures' etiologies

In [ ]:
# Function to categorize values
def categorize(val):
    return 'no Presence' if 'non' in val else 'Presence'

cancer_types = ['Breast-cancer', 'Lung-AdenoCa', 'ColoRect-AdenoCa', 'Liver-HCC']

# Create a figure with 5 subplots
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(6, 6.5), sharex=True)

# Use one general y-label for all subplots
fig.text(-0.01, 0.5, 'Proportion', va='center', rotation='vertical', fontsize=12)

for i, cancer_type in enumerate(cancer_types):
    
    ## Prepare the data
    df_frac = dict_allFrac[cancer_type].copy()

    ### Let's check how the distribution of different labeling results
    labels = ['label_Apobec', 'label_MMR', 'label_Tobacco', 'label_UV', 'label_POLE', 'label_ClockLike', 'label_BER', 'label_Platinum']
    new_labels = ['Apobec', 'MMR', 'Tobacco', 'UV', 'POLE', 'ClockLike', 'BER', 'Platinum']
    df_labels = df_frac[labels]
    
    # Apply the categorization function to each element in the DataFrame
    df_transformed = df_labels.map(categorize)
    
    # Calculate the proportions
    proportions = df_transformed.apply(lambda x: x.value_counts(normalize=True)).fillna(0)
    
    # Transpose for plotting
    proportions = proportions.T
    
    # Reset the index with the given values
    df_reset = proportions.set_index(pd.Index(new_labels))

    ## Plotting
    df_reset.plot(kind='bar', stacked=True, ax=axes[i], legend=False)
    axes[i].set_ylabel('')
    axes[i].set_xticklabels(df_reset.index, 
                            rotation=45, ha='right') 
    axes[i].set_title(f'In {cancer_type} sample', loc='left')
    
    # Calculate and add ratio labels to the top of each stacked bar
    for label in new_labels:
        total_presence = df_reset.loc[label, 'Presence']
        total_non_presence = df_reset.loc[label, 'no Presence']
        total = total_presence + total_non_presence
        if total != 0:
            ratio = total_presence / total_non_presence
        else:
            ratio = 0
        axes[i].text(new_labels.index(label), total, f'{ratio:.2f}', ha='center', va='top', #y=0.95
                    )

# Adjust x-ticks for one of the axes (they are shared)
axes[0].set_xticklabels(df_reset.index, rotation=45, ha='right')

# Create a shared legend
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper left', bbox_to_anchor=(0.75, 1.0), ncol=1)

# Add a main title to the figure
fig.suptitle('Proportions of Presence of SBS signatures, Hartwig', x=0.35, fontsize=12)

# Adjust layout
plt.tight_layout()

## save the figure
plt.savefig("Hartwig_Proportions of Presence of SBS signatures in cancer samples_RatioShown_exper01.pdf",
            format="pdf",
            dpi=999,
            bbox_inches="tight")

plt.show()


### Draw the Density of SBS signaturtes siginals in samples

In [ ]:
import seaborn as sns
print(sns.__version__)

cancer_types = ['Breast-cancer', 'Lung-AdenoCa', 'ColoRect-AdenoCa', 'Liver-HCC']

### define the color pannel
color_palette_signal = {
    "signal_Apobec": "#1f77b4",      # Blue
    "signal_MMR": "#ff7f0e",         # Orange
    "signal_Tobacco": "#2ca02c",     # Green
    "signal_UV": "#d62728",          # Red
    "signal_POLE": "#9467bd",        # Purple
    "signal_ClockLike": "#8c564b",   # Brown
    "signal_BER": "#e377c2",         # Magenta
    "signal_Platinum": "#17becf",    # Cyan
}

# Create a figure with 5 subplots
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(6, 6.5), sharex=True)

# Use one general y-label for all subplots
fig.text(-0.01, 0.5, 'Density of Signal', va='center', rotation='vertical', fontsize=12)

for i, cancer_type in enumerate(cancer_types):
    
    ## Prepare the data
    df_frac = dict_allFrac[cancer_type].copy()

    ### Let's check how the distribution of different labeling results
    labels = ['signal_Apobec', 'signal_MMR', 'signal_Tobacco', 'signal_UV', 'signal_POLE', 'signal_ClockLike', 'signal_BER', 'signal_Platinum']
    df_labels = df_frac[labels]
    
    # Reshape the DataFrame to long format
    df_long = df_labels.melt(var_name='Signature_Class', value_name='Density')

    ## Plotting
    sns.boxplot(x='Signature_Class', y='Density', data=df_long, 
                ax=axes[i], 
                hue='Signature_Class', palette=color_palette_signal, legend=False
               )
    axes[i].set_ylabel('')
    axes[i].set_xlabel('') 
    axes[i].set_xticks(range(len(df_reset.index)))
    axes[i].set_xticklabels(df_reset.index, rotation=45, ha='right') 
    axes[i].set_title(f'In {cancer_type} sample', loc='left')
    #axes[i].legend().remove() 
    
# Adjust x-ticks for one of the axes (they are shared)
# Set x-tick positions
axes[0].set_xticks(range(len(df_reset.index)))
axes[0].set_xticklabels(df_reset.index, rotation=45, ha='right')

# Create a shared legend
handles, labels = axes[0].get_legend_handles_labels()
# fig.legend(handles, labels, loc='lower left', bbox_to_anchor=(1.0, 0.5), ncol=1)

# Add a main title to the figure
fig.suptitle('Density of of SBS signatures, Hartwig', fontsize=12, x=0.3)

# Adjust layout
plt.tight_layout()

## save the figure
plt.savefig("Hartwig_Density of Presence of SBS signatures in cancer samples_exper01.pdf"
                ,format="pdf"
                ,dpi=999
                ,bbox_inches="tight")
plt.show()


### Draw the UpSet plot to show one sample can show exposures to signatures attributed to different etiolohies

In [ ]:
from upsetplot import from_contents
from upsetplot import UpSet
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

cancer_types = ['Breast-cancer', 'Lung-AdenoCa', 'ColoRect-AdenoCa', 'Liver-HCC']

# Draw the UpSet plot
for i, cancer_type in enumerate(cancer_types):
    
    ## Prepare the data
    df_frac = dict_allFrac[cancer_type].copy()
    df_labels = from_contents({'Apobec': df_frac[df_frac['label_Apobec'] == 'Apobec'].index.tolist(),
                               'MMR': df_frac[df_frac['label_MMR'] == 'MMR'].index.tolist(),
                               'Tobacco': df_frac[df_frac['label_Tobacco'] == 'Tobacco'].index.tolist(),
                               'UV': df_frac[df_frac['label_UV'] == 'UV'].index.tolist(),
                               'POLE': df_frac[df_frac['label_POLE'] == 'POLE'].index.tolist(),
                               'ClockLike': df_frac[df_frac['label_ClockLike'] == 'ClockLike'].index.tolist(),
                               'BER': df_frac[df_frac['label_BER'] == 'BER'].index.tolist(),
                               'Platinum': df_frac[df_frac['label_Platinum'] == 'Platinum'].index.tolist(),
                              })

    ## Plotting
    fig = plt.figure(figsize=(8, 4))
    ax_dict = UpSet(df_labels, 
                    subset_size='count', 
                    show_counts=True, 
                    totals_plot_elements=5, )
    ax_dict.plot()
    plt.suptitle(f'Co-presence of mutational signatures aetiologies, in {cancer_type} sample, Hartwig', 
                 fontsize=18, x=0.35, y=0.95, ha='center')
    
    ## save the figure
    plt.savefig(f"Co-presence of mutational signatures aetiologies, in {cancer_type} sample_Hartwig_exper01.pdf"
                ,format="pdf"
                ,dpi=999
                ,bbox_inches="tight")
    plt.show()



### Draw the UpSet plot to show the presence of APOBEC signatures (SBS2 and SBS13）in cancer samples

In [ ]:
from upsetplot import from_contents
from upsetplot import UpSet

cancer_types = ['Breast-cancer', 'Lung-AdenoCa', 'ColoRect-AdenoCa', 'Liver-HCC']
    
# Draw the UpSet plot
for i, cancer_type in enumerate(cancer_types):

    ## Prepare the data
    df_frac = dict_allFrac[cancer_type].copy()
    df_labels = from_contents({'SBS2': df_frac[df_frac['SBS2'] != 0].index.tolist(),
                               'SBS13': df_frac[df_frac['SBS13'] != 0].index.tolist(),
                               'no-Apobec':df_frac[(df_frac['SBS13'] == 0) & (df_frac['SBS2'] == 0)].index.tolist()
                              })

    ## Plotting
    fig = plt.figure(figsize=(5, 5))
    sns.set_style('ticks')
    ax_dict = UpSet(df_labels, 
                    subset_size='count', 
                    show_counts=True, 
                    totals_plot_elements=5, )
    ax_dict.plot()
    plt.suptitle(f'Presence of Apobec Signatures, in {cancer_type}, Hartwig', 
                 fontsize=12, x=0.45, y=0.95, ha='center')
    
    ## save the figure
    plt.savefig(f"Presence of Apobec Signatures, in {cancer_type}_Hartwig_exper01.pdf"
                ,format="pdf"
                ,dpi=999
                ,bbox_inches="tight")
    plt.show()



### Draw the multiple labeling heatmap of the signatures exposures 

In [ ]:

import warnings
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages


def plot_clustermap(cancer_type, df_sample, multi_label_info, flag=True):
    
    '''
    cancer_type: To specify which cancer data that is applied
    df_sample: The Fraction data of signatures in samples
    multi_label_info: The lables of samples that used to specify the groups of samples, should be a pandas dataframe
    flag: Perform the clustering or not, default is not
    '''
    
    # Ensure multi_label_info is a DataFrame
    if not isinstance(multi_label_info, pd.DataFrame):
        raise ValueError("multi_label_info must be a pandas DataFrame")
    
    # Specific color mappings for each label
    color_mappings = {
    'label_Apobec': {'Apobec': "#1f77b4", 'nonApobec': 'lightgrey'},
    'label_MMR': {'MMR': "#ff7f0e", 'nonMMR': 'lightgrey'},
    'label_Tobacco': {'Tobacco': "#2ca02c", 'nonTobacco': 'lightgrey'},
    'label_UV': {'UV': "#d62728", 'nonUV': 'lightgrey'},
    'label_POLE': {'POLE': "#9467bd", 'nonPOLE': 'lightgrey'},
    'label_ClockLike': {'ClockLike': "#8c564b", 'nonClockLike': 'lightgrey'},
    'label_BER': {'BER': "#e377c2", 'nonBER': 'lightgrey'},
    'label_Platinum': {'Platinum': "#17becf", 'nonPlatinum': 'lightgrey'},  
    }

    # Map labels to specific colors and combine
    label_color_dfs = []
    for label in multi_label_info.columns:
        if label in color_mappings:
            color_labels = multi_label_info[label].map(color_mappings[label])
            label_color_dfs.append(color_labels)

    combined_label_colors = pd.concat(label_color_dfs, axis=1)

    ## Prepare the custom legend
    color_mappings_ss = {
    "Apobec": "#1f77b4",      # Blue
    "MMR": "#ff7f0e",         # Orange
    "Tobacco": "#2ca02c",     # Green
    "UV": "#d62728",          # Red
    "POLE": "#9467bd",        # Purple
    "ClockLike": "#8c564b",   # Brown
    "BER": "#e377c2",         # Magenta
    "Platinum": "#17becf",    # Cyan
    }

    # Create a list of patches to add to the legend
    patches = [mpatches.Patch(color=color, label=label) for label, color in color_mappings_ss.items()]

    # Draw the clustermap
    sns.set(font_scale=0.7)
    g = sns.clustermap(df_sample.T, method='average', col_colors=combined_label_colors, cmap="coolwarm", 
                        vmax=1, col_cluster=flag, row_cluster=True, figsize=(12, 9))
    # Access the axes of the clustermap
    ax = g.ax_heatmap
    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize=6)  # Adjust fontsize as needed
    ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize=6)  # Adjust fontsize as needed
    
    # Set x-label and y-label
    g.ax_heatmap.set_xlabel("Samples", fontsize=12)
    g.ax_heatmap.set_ylabel("Mutational Signatures", fontsize=12)
    
    plt.title('Heatmap of Signatures exposures fractions in {0} sample, Hartwig'.format(cancer_type), 
              fontsize=14, loc="left", y=1.05)
    
    # Add the custom legend to the clustermap figure
    for ax in g.fig.axes:  # Loop through the axes in the clustermap
        if ax.get_label() == 'legend':  # Find the legend axis
            ax.remove()  # Remove the existing legend axis

    # Add the custom legend patches
    g.ax_heatmap.legend(handles=patches, loc='upper left', bbox_to_anchor=(-0.15, 1.44), ncol=len(color_mappings) // 8, fontsize='large')

    # save the file
    plt.savefig('Heatmap of Signatures exposures fractions in {0} sample, Hartwig_exper01.pdf'.format(cancer_type)
                ,format="pdf"
                ,dpi=999
                ,bbox_inches="tight")
    
    # Append the plot to the PDF file
    pdf.savefig(dpi=999, bbox_inches="tight")
    plt.show()
    plt.close()

## Using the mutational signatures Fraction data
dict_DF = dict_allFrac ## The mutational signatures fraction data
cancer_types = ['Breast-cancer', 'Lung-AdenoCa',  'ColoRect-AdenoCa', 'Liver-HCC']
labels = ['label_Apobec', 'label_MMR', 'label_Tobacco', 'label_UV', 'label_POLE', 'label_ClockLike', 'label_BER', 'label_Platinum']

## Draw the plots
with PdfPages('Heatmap of Signatures exposures fractions in cancer samples_Hartwig_all_exper01.pdf') as pdf:
    for cancer_type, df_sample in dict_DF.items():
        ## get the multiple labels
        multi_label_info = df_sample[labels]
        df_sample = df_sample[Sigs]
        plot_clustermap(cancer_type, df_sample, multi_label_info)


### Groupping the signatures of same etiology class, then normalize the exposures to fractions per each sample

In [ ]:
filenames = ['Hartwig_BRCA_Assignment_Solution_Activities_PCAWGref.txt',
             'Hartwig_LUAD_Assignment_Solution_Activities_PCAWGref.txt',
             'Hartwig_COREAD_Assignment_Solution_Activities_PCAWGref.txt',
             'Hartwig_HCC_Assignment_Solution_Activities_PCAWGref.txt'
             ]
cancer_types = ['Breast-cancer', 'Lung-AdenoCa', 'ColoRect-AdenoCa', 'Liver-HCC']

dict_allExpo = {} ## dict to accept prepared data

for cancer_type, filename in zip(cancer_types, filenames):
    ## load the file
    df = pd.read_csv(filename, sep='\t')

    df=df.rename(columns={'Samples': 'clinic_ID'})
    df=df.set_index('clinic_ID')
    df['Cancer_Types'] = cancer_type

    ## Move 'Cancer_Type' to the first position
    column_order = ['Cancer_Types'] + [col for col in df if col != 'Cancer_Types']
    df = df[column_order]
    
    ## Take the signatures
    Sigs = df.columns[1:]
    d0 = df.copy()
    
    ## 
    df_raw = d0.drop(['Cancer_Types'], axis = 1)
    df_raw = df_raw.loc[~(df_raw == 0).all(axis=1)] # drop rows if all of their values equal to 0

    ## Calculate the Expotions of mutational signatuters exposures in samples
    #total_counts = df_raw.sum(axis=1)
    #df_allExpo = df_raw.div(total_counts, axis=0)
    
    df_allExpo = df_raw.copy()

    ## multi-labeling of samples according to the existences of certain class of signatures
    # Apobec signaitures
    df_allExpo['label_Apobec'] = df_allExpo.apply(lambda row: 'nonApobec' if (row['SBS2'] == 0 and row['SBS13'] == 0) 
                                                  else 'Apobec', axis=1)
    df_allExpo['signal_Apobec'] = df_allExpo['SBS2'] + df_allExpo['SBS13']

    # Mismatch Repair signatures
    df_allExpo['label_MMR'] = df_allExpo.apply(lambda row: 'nonMMR' if (row['SBS6'] == 0 
                                                                        and row['SBS14'] == 0 
                                                                        and row['SBS15'] == 0 
                                                                        and row['SBS20'] == 0 
                                                                        and row['SBS21'] == 0 
                                                                        and row['SBS26'] == 0 
                                                                        and row['SBS44'] == 0) 
                                                     else 'MMR', axis=1)
    df_allExpo['signal_MMR'] = df_allExpo['SBS6'] + df_allExpo['SBS14'] + df_allExpo['SBS15'] + df_allExpo['SBS20'] + df_allExpo['SBS21'] + df_allExpo['SBS26'] + df_allExpo['SBS44'] 

    # Tobacco signatures
    df_allExpo['label_Tobacco'] = df_allExpo.apply(lambda row: 'nonTobacco' if (row['SBS4'] == 0 
                                                                                and row['SBS29'] == 0) 
                                                   else 'Tobacco', axis=1)
    df_allExpo['signal_Tobacco'] = df_allExpo['SBS4'] + df_allExpo['SBS29']

    # UV signatures
    df_allExpo['label_UV'] = df_allExpo.apply(lambda row: 'nonUV' if (row['SBS7a'] == 0 
                                                                      and row['SBS7b'] == 0 
                                                                      and row['SBS7c'] == 0 
                                                                      and row['SBS7d'] == 0 
                                                                      and row['SBS38'] == 0) 
                                              else 'UV', axis=1)
    df_allExpo['signal_UV'] = df_allExpo['SBS7a'] + df_allExpo['SBS7b'] + df_allExpo['SBS7c'] + df_allExpo['SBS7d'] + df_allExpo['SBS38']

    # POLE signatures
    df_allExpo['label_POLE'] = df_allExpo.apply(lambda row: 'nonPOLE' if (row['SBS10a'] == 0 
                                                                        and row['SBS10b'] == 0) 
                                                 else 'POLE', axis=1)
    df_allExpo['signal_POLE'] = df_allExpo['SBS10a'] + df_allExpo['SBS10b']

    # Clock-likesignatures
    df_allExpo['label_ClockLike'] = df_allExpo.apply(lambda row: 'nonClockLike' if (row['SBS1'] == 0) 
                                                     else 'ClockLike', axis=1)
    df_allExpo['signal_ClockLike'] = df_allExpo['SBS1']

    # Base_Excision_Repair, BER
    df_allExpo['label_BER'] = df_allExpo.apply(lambda row: 'nonBER' if (row['SBS30'] == 0 
                                                                        and row['SBS36'] == 0) 
                                               else 'BER', axis=1)
    df_allExpo['signal_BER'] = df_allExpo['SBS18'] + df_allExpo['SBS30'] + df_allExpo['SBS36']

    # Platinum Treatment
    df_allExpo['label_Platinum'] = df_allExpo.apply(lambda row: 'nonPlatinum' if (row['SBS31'] == 0 
                                                                                  and row['SBS35'] == 0) 
                                                    else 'Platinum', axis=1)
    df_allExpo['signal_Platinum'] = df_allExpo['SBS31'] + df_allExpo['SBS35']

    ## Take a look of the data
    df_allExpo['Cancer_Types'] = df['Cancer_Types']
    
    dict_allExpo[cancer_type] = df_allExpo

dict_allExpo
# Sigs

In [ ]:
## The etiology info of SBS signatures
aetilogy_info = {
    'SBS1': 'Clock-like', # Deamination of 5-methylcytosine
    
    'SBS2': 'APOBEC', # APOBEC activity
    'SBS13': 'APOBEC',  # APOBEC activity
    
    'SBS3': 'HR', # Defective HR DNA repaire; BRCA1/2 mutation
    
    'SBS4': 'Tobacco', # Smoking
    'SBS29': 'Tobacco', # Smoking
    
    'SBS6': 'MMR', # Defective DNA mismatch repair
    'SBS14': 'MMR', # POLE mutation and mismatch repair deficiency
    'SBS15': 'MMR', # Defective DNA mismatch repair
    'SBS20': 'MMR', # POLD1 mutation and mismatch repair deficiency
    'SBS21': 'MMR', # Defective DNA mismatch repair
    'SBS26': 'MMR', # Defective DNA mismatch repair
    'SBS44': 'MMR', # Defective DNA mismatch repair
    
    'SBS7a':'UV_light', # Ultraviolet light exposure
    'SBS7b':'UV_light', # Ultraviolet light exposure
    'SBS7c':'UV_light', # Ultraviolet light exposure
    'SBS7d':'UV_light', # Ultraviolet light exposure
    'SBS38':'UV_light', # Indirect effect of ultravioleat light
    
    'SBS10a': 'POLE', # POLE mutation
    'SBS10b': 'POLE', # POLE mutation
    
    'SBS30': 'BER', # Defective base excision repair; NTHL1 mutation
    'SBS36': 'BER', # Defective base excision repair; MUTYH mutation
    
    'SBS31': 'Platinum_treat', # Platinum treatment
    'SBS35': 'Platinum_treat', # Platinum treatment
    
    'SBS9': 'POLEerase', # In part, POLEerase activity
    'SBS11': 'Temozolomide_treat', # Temozolomide treatment
    'SBS18': 'Reactive_oxygen', # Reactive oxygen species
    'SBS22': 'Aristolochic_acid', # Aristolochic acid exposure
    'SBS24': 'Aflatoxin_expo', # Aflatoxin exposure
    'SBS25': 'Chemotherapy', # Chemotherapy
    'SBS32': 'Azathioprine_treat', # Azathioprine treatment
    'SBS42': 'Haloalkane_expo', # Haloalkane exposure
    
    'SBS5': 'unknown', 
    'SBS8': 'unknown',
    'SBS12': 'unknown',
    'SBS16': 'unknown', 
    'SBS17a': 'unknown',
    'SBS17b': 'unknown',
    'SBS19': 'unknown', 
    'SBS23': 'unknown', 
    'SBS28': 'unknown',
    'SBS33': 'unknown', 
    'SBS34': 'unknown', 
    'SBS37': 'unknown', 
    'SBS39': 'unknown',
    'SBS40': 'unknown',
    'SBS41': 'unknown',
    
    'SBS27':'Artefact', 
    'SBS43':'Artefact', 
    'SBS45':'Artefact',
    'SBS46':'Artefact',
    'SBS47':'Artefact',
    'SBS48':'Artefact',
    'SBS49':'Artefact',
    'SBS50':'Artefact',
    'SBS51':'Artefact',
    'SBS52':'Artefact',
    'SBS53':'Artefact',
    'SBS54':'Artefact',
    'SBS55':'Artefact',
    'SBS56':'Artefact',
    'SBS57':'Artefact',
    'SBS58':'Artefact',
    'SBS59':'Artefact',
    'SBS60':'Artefact', 
    'SBS84':'Artefact',
    'SBS85':'Artefact', 
}

# aetilogy_info

In [ ]:
### Average the signaturers of same etiology group
cancer_types = ['Breast-cancer', 'Lung-AdenoCa',  'ColoRect-AdenoCa', 'Liver-HCC']

# First, filter out the signatures that don't need to be merged
signatures_to_merge = {k: v for k, v in aetilogy_info.items() if v in ['Clock-like', 'APOBEC', 'HR', 'Tobacco', 'MMR', 'UV_light', 'POLE', 'BER', 
                                                                'Platinum_treat', 'POLEerase', 'Temozolomide_treat', 'Reactive_oxygen',
                                                                'Aristolochic_acid', 'Aflatoxin_expo', 'Chemotherapy', 'Azathioprine_treat', 
                                                                'Haloalkane_expo']}

# Then, extract these signatures into a new DataFrame, and merge them

dict_allExpo_g = {}
for cancer_type, df_Expo in dict_allExpo.items(): ## The mutational signatures Expotion data
    df_to_merge = df_Expo[list(signatures_to_merge.keys())]

    # Now, merge these signatures by calculating their mean values according to their group
    merged_df = df_to_merge.T.groupby(signatures_to_merge).mean()
    merged_df_T = merged_df.T

    # Next, extract columns dont need to be merged
    nonCols = [col for col in df_Expo.columns if col not in signatures_to_merge.keys()]
    df_not_merged = df_Expo[nonCols]

    # Combined the merged and non-merged data
    combined_df = pd.concat([merged_df_T, df_not_merged], axis=1)

    combined_df.columns  # Display the first few rows of the final DataFrame

    df_Expo_g = combined_df.copy()
    dict_allExpo_g[cancer_type] = df_Expo_g
    
Sigs_g = ['Clock-like', 'APOBEC', 'HR', 'Tobacco', 'MMR', 'UV_light', 'POLE', 'BER', 'Platinum_treat', 'POLEerase', 'Temozolomide_treat', 'Reactive_oxygen',
          'Aristolochic_acid', 'Aflatoxin_expo', 'Chemotherapy', 'Azathioprine_treat', 'Haloalkane_expo', 
          'SBS5', 'SBS8', 'SBS12', 'SBS16', 'SBS17a', 'SBS17b', 'SBS19', 'SBS23', 'SBS28', 'SBS33', 'SBS34', 'SBS37', 'SBS39', 'SBS40', 'SBS41',
          'SBS27', 'SBS43', 'SBS45', 'SBS46', 'SBS47', 'SBS48', 'SBS49', 'SBS50', 'SBS51', 'SBS52', 'SBS53', 'SBS54', 'SBS55', 'SBS56', 'SBS57', 
          'SBS58','SBS59','SBS60', 'SBS84','SBS85']

dict_allExpo_g
dict_allExpo_g['Breast-cancer']


In [ ]:
### Now, based on the average exposures, generate the frac Data
dict_allFrac_g = {}
cancer_types = ['Breast-cancer', 'Lung-AdenoCa',  'ColoRect-AdenoCa', 'Liver-HCC']

for cancer_type in cancer_types:
    
    df_ = dict_allExpo_g[cancer_type]
    
    df_raw = df_[Sigs_g]
    ## Calculate the Fractions of mutational signatuters exposures in samples
    total_counts = df_raw.sum(axis=1)
    df_allFrac_g = df_raw.div(total_counts, axis=0)
    
    ## multi-labeling of samples according to the existences of certain class of signatures

    # Apobec signaitures
    df_allFrac_g['label_Apobec'] = df_allFrac_g.apply(lambda row: 'nonApobec' if (row['APOBEC'] == 0) else 'Apobec', axis=1)
    df_allFrac_g['signal_Apobec'] = df_allFrac_g['APOBEC']

    # Mismatch Repair signatures
    df_allFrac_g['label_MMR'] = df_allFrac_g.apply(lambda row: 'nonMMR' if (row['MMR'] == 0) else 'MMR', axis=1)
    df_allFrac_g['signal_MMR'] = df_allFrac_g['MMR']
    
    # Tobacco signatures
    df_allFrac_g['label_Tobacco'] = df_allFrac_g.apply(lambda row: 'nonTobacco' if (row['Tobacco'] == 0) else 'Tobacco', axis=1)
    df_allFrac_g['signal_Tobacco'] = df_allFrac_g['Tobacco']

    # UV signatures
    df_allFrac_g['label_UV'] = df_allFrac_g.apply(lambda row: 'nonUV' if (row['UV_light'] == 0) else 'UV', axis=1)
    df_allFrac_g['signal_UV'] = df_allFrac_g['UV_light']
    
    # POLE signatures
    df_allFrac_g['label_POLE'] = df_allFrac_g.apply(lambda row: 'nonPOLE' if (row['POLE'] == 0) else 'POLE', axis=1)
    df_allFrac_g['signal_POLE'] = df_allFrac_g['POLE']

    # Clock-likesignatures
    df_allFrac_g['label_ClockLike'] = df_allFrac_g.apply(lambda row: 'nonClockLike' if (row['Clock-like'] == 0) else 'ClockLike', axis=1)
    df_allFrac_g['signal_ClockLike'] = df_allFrac_g['Clock-like']

    # Base_Excision_Repair, BER
    df_allFrac_g['label_BER'] = df_allFrac_g.apply(lambda row: 'nonBER' if (row['BER'] == 0) else 'BER', axis=1)
    df_allFrac_g['signal_BER'] = df_allFrac_g['BER']

    # Platinum Treatment
    df_allFrac_g['label_Platinum'] = df_allFrac_g.apply(lambda row: 'nonPlatinum' if (row['Platinum_treat'] == 0) else 'Platinum', axis=1)
    df_allFrac_g['signal_Platinum'] = df_allFrac_g['Platinum_treat']
    
    ## Take a look of the data
    df_allFrac_g['Cancer_Types'] = cancer_type
    
    dict_allFrac_g[cancer_type] = df_allFrac_g

dict_allFrac_g

### Calculate distance between Samples regarding different distance metrics, for normalized exposures, grouped + normalized exposures and raw exposures

In [ ]:
### Load the distance matrix that used for EMD calculation

## Cosine distance between Signatures
Sig_cosine = pd.read_csv('Cosine distance between SBS signatures_exper01_Hartwig.csv', index_col='Unnamed: 0')
Sig_cosine

## Hybrid distance between Signatures
Sig_hybrid = pd.read_csv('Hybrid distance between SBS signatures_exper01_Hartwig.csv', index_col='Unnamed: 0')
Sig_hybrid

## Etiological distance between signature with third disttance as 0.5
Sig_func = pd.read_csv('Etiology Distance with thirdDist=0.50 distance between SBS signatures_exper01_Hartwig.csv', index_col='Unnamed: 0')
Sig_func


In [ ]:
### Extract pure SBS signatures data

cancer_types = ['Breast-cancer', 'Lung-AdenoCa',  'ColoRect-AdenoCa', 'Liver-HCC']

dict_allExpo_ss = {} # dict for pure signatures data for each cancer type, Exposure data
dict_allFrac_ss = {} # dict for pure signatures data for each cancer type, Fraction data
dict_allFrac_g_ss = {} # dict for pure signatures (grouped) data for each cancer type, Fraction data

## Extract signatures for each cancer type
for cancer_type in cancer_types:

    ## for pure signatures, Exposure data
    df_expo = dict_allExpo[cancer_type]
    df_expo_ss = df_expo[Sigs]
    dict_allExpo_ss[cancer_type]=df_expo_ss
    
    ## for pure signatures, normalized data
    df_frac = dict_allFrac[cancer_type]
    df_frac_ss = df_frac[Sigs]
    dict_allFrac_ss[cancer_type]=df_frac_ss
    
    ## for grouped signatures, normalized data
    df_frac_g = dict_allFrac_g[cancer_type]
    df_frac_g_ss = df_frac_g[Sigs_g]
    dict_allFrac_g_ss[cancer_type]=df_frac_g_ss

## Take a look of the data
dict_allExpo_ss # raw exposure data
dict_allFrac_ss # normalized data
dict_allFrac_g_ss # grouped fractioned data


In [ ]:
### Calculate the distances between samples, regarding the fraction of exposure of signatures

from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cosine

from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

## define the function of emd calculation
def emdCosine(p, q, SigDistMat=Sig_cosine):
    emd_dist = emd(np.ascontiguousarray(p, dtype=np.float64),
                   np.ascontiguousarray(q, dtype=np.float64),
                   np.ascontiguousarray(SigDistMat, dtype=np.float64))
    return emd_dist

def emdFunc(p, q, SigDistMat=Sig_func):
    emd_dist = emd(np.ascontiguousarray(p, dtype=np.float64),
                   np.ascontiguousarray(q, dtype=np.float64),
                   np.ascontiguousarray(SigDistMat, dtype=np.float64))
    return emd_dist

def emdHybrid(p, q, SigDistMat=Sig_hybrid):
    emd_dist = emd(np.ascontiguousarray(p, dtype=np.float64),
                   np.ascontiguousarray(q, dtype=np.float64),
                   np.ascontiguousarray(SigDistMat, dtype=np.float64))
    return emd_dist

## Use only mutational signatures fractions data
dict_DFs = dict_allFrac_ss.copy() ## the mutational signatures data
cancer_types = ['Breast-cancer', 'Lung-AdenoCa',  'ColoRect-AdenoCa', 'Liver-HCC']
dict_distDFs = {} ## The dict to accept the distance matrix of cancer samples 
## Metrics of the distance type
Metrics = [euclidean, cosine, emdCosine, emdFunc, emdHybrid]

## Calculate distance between samples, using different metrices
start_time = time.time() # The start time

for cancer_type in cancer_types:
    
    ## list to collect results
    distDFs = []
    d0 = dict_DFs[cancer_type]
    
    ## use varied metrices for distnace calculation
    for Metric in Metrics:
        dists = pdist(d0, metric=Metric) # pairwise distance calculation
        dist_mat = squareform(dists) # convert the condensed distance matrix into a full distance matrix
        dist_df = pd.DataFrame(dist_mat)
        dist_df.columns, dist_df.index = d0.index, d0.index ## assign colnames and row names
        
        ## Collect the distance matrix
        distDFs.append(dist_df)
        
    ## Collect the distance of distance matrix
    dict_distDFs[cancer_type] = distDFs

    ## The end time
    end_time = time.time()
    print("Elapsed time:", (end_time - start_time)/60, f"minutes for {cancer_type}")


In [ ]:
### Calculate the distances between samples, regarding the Fraction of exposures of signatures (grouped)

dict_DFs = dict_allFrac_g_ss.copy() ## The mutational signatures data
cancer_types = ['Breast-cancer', 'Lung-AdenoCa',  'ColoRect-AdenoCa', 'Liver-HCC']

dict_distDFs_g = {} ## The dict to accept the distance matrix of cancer samples 
## Metrics of the distance type
Metrics = [euclidean, cosine]

## Calculate distance between samples, using different metrices
start_time = time.time() # The start time

for cancer_type in cancer_types:
    
    ## list to collect results
    distDFs = []
    d0 = dict_DFs[cancer_type]
    
    ## use varied metrices for distnace calculation
    for Metric in Metrics:
        dists = pdist(d0, metric=Metric) # pairwise distance calculation
        dist_mat = squareform(dists) # convert the condensed distance matrix into a full distance matrix
        dist_df = pd.DataFrame(dist_mat)
        dist_df.columns, dist_df.index = d0.index, d0.index ## assign colnames and row names
        
        ## Collect the distance matrix
        distDFs.append(dist_df)
        
    ## Collect the distance of distance matrix
    dict_distDFs_g[cancer_type] = distDFs

    ## The end time
    end_time = time.time()
    print("Elapsed time:", (end_time - start_time)/60, f"minutes for {cancer_type}")


In [ ]:
### Calculate the distances between samples, regarding the raw exposures of signatures

from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

dict_DFs = dict_allExpo_ss.copy() ## The mutational signatures data
cancer_types = ['Breast-cancer', 'Lung-AdenoCa',  'ColoRect-AdenoCa', 'Liver-HCC']
dict_distDFs_expo = {} ## The dict to accept the distance matrix of cancer samples 
## Metrics of the distance type
Metrics = [euclidean, #cosine
          ]

## Calculate distance between samples, using different metrices
start_time = time.time() # The start time

for cancer_type in cancer_types:
    
    ## list to collect results
    distDFs = []
    d0 = dict_DFs[cancer_type]
    
    ## use varied metrices for distnace calculation
    for Metric in Metrics:
        dists = pdist(d0, metric=Metric) # pairwise distance calculation
        dist_mat = squareform(dists) # convert the condensed distance matrix into a full distance matrix
        dist_df = pd.DataFrame(dist_mat)
        dist_df.columns, dist_df.index = d0.index, d0.index ## assign colnames and row names
        
        ## Collect the distance matrix
        distDFs.append(dist_df)
        
    ## Collect the distance of distance matrix
    dict_distDFs_expo[cancer_type] = distDFs

    ## The end time
    end_time = time.time()
    print("Elapsed time:", (end_time - start_time)/60, f"minutes for {cancer_type}")


In [ ]:
## Using dicts to hold the distance matrixs with types of distance as keys
cancer_types = ['Breast-cancer', 'Lung-AdenoCa',  'ColoRect-AdenoCa', 'Liver-HCC']

## Defining names of the metrices, for the normal signatures, fraction
Names = ['Euclidean', 'Cosine', 'cEMD', 'eEMD', 'hEMD']
## Names of the metrices, for the grouped signatures, fraction
Names_g = ['gEuclidean', 'gCosine']
## Names of the metrices, for the normal signatures, exposure
Names_expo = ['rEuclidean']


## dict to accept the dicts of distance matrix
dict_distDFs_dict = {} 
for cancer_type in cancer_types:
    
    ## for normal signatures, frequency
    distDFs = dict_distDFs[cancer_type]
    distDFs_dict = dict(zip(Names, distDFs))
    
    ## for grouped signatures, frequency
    distDFs_g = dict_distDFs_g[cancer_type]
    distDFs_dict_g = dict(zip(Names_g, distDFs_g))

    ## for normal signatures, raw exposure
    distDFs_expo = dict_distDFs_expo[cancer_type]
    distDFs_dict_expo = dict(zip(Names_expo, distDFs_expo))

    ## Put them together
    distDFs_dict.update(distDFs_dict_g)
    distDFs_dict.update(distDFs_dict_expo)

    ## Collect the results
    dict_distDFs_dict[cancer_type] = distDFs_dict


In [ ]:
### Save the dicts of the distance matrix for each cancer type
import pickle

cancer_types = ['Breast-cancer', 'Lung-AdenoCa',  'ColoRect-AdenoCa', 'Liver-HCC']

for cancer_type in cancer_types:
    
    distDFs_dict = dict_distDFs_dict[cancer_type]
    
    ## Write the dict to a pickle file
    with open(f'Hartwig_all_distDFs_dict_for_{cancer_type}_Sample_exper01.pickle', 'wb') as handle:
        pickle.dump(distDFs_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    ## Check if the files were saved correctly
    with open(f'Hartwig_all_distDFs_dict_for_{cancer_type}_Sample_exper01.pickle', 'rb') as handle:
        distDFs_dict_xx = pickle.load(handle)
    print(distDFs_dict_xx)

### The file f'Hartwig_all_distDFs_dict_for_{cancer_type}_Sample_v1_new_eEMD.pickle', will be used for Clustering analysis

### Also save the file of 'dict_allFrac', which including the fraction data of not grouped SBS signatures, 
### and multiple labeling info for all samples from the 5 cancer type, which used cancer name as keys

# write 'dict_allFrac' to a pickle file
with open('Hartwig_SigFrac_dict_for_all_cancer_Samples_exper01.pickle', 'wb') as handle:
    pickle.dump(dict_allFrac, handle, protocol=pickle.HIGHEST_PROTOCOL)
